In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

In [3]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora', transform=NormalizeFeatures())

# Access the data (Cora has only one graph, so we can access it via dataset[0])
data = dataset[0]

In [4]:
print(f'Dataset: {dataset}:')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'\nData object: {data}')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.has_isolated_nodes()}')
print(f'Contains self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data object: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [5]:
dataset.y[dataset.train_mask].shape

torch.Size([140])

In [8]:
from model import GAT
gat_model = GAT(in_channels=dataset.num_features, 
                hid_units=[8, 8],  # Hidden units for each GAT layer
                n_heads=[8, 8],  # Number of attention heads per layer
                dropout=0.6, 
                num_classes=dataset.num_classes,
                residual=True)

In [7]:
# out_channels=dataset.num_classes,

In [9]:
gat_model

GAT(
  (attentions): ModuleList(
    (0): GATConv(1433, 8, heads=8)
  )
  (hidden_attentions): ModuleList(
    (0): GATConv(64, 8, heads=8)
  )
  (out_attention): GATConv(64, 8, heads=1)
  (dense): Linear(in_features=8, out_features=7, bias=True)
)

In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from sklearn.metrics import accuracy_score

# Function to train the GAT model
def train(model, optimizer, data, epochs=200):
    model.train()  # Set the model to training mode

    for epoch in range(epochs):
        optimizer.zero_grad()  # Clear gradients from the previous step
        out = model(data)  # Forward pass
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])  # Compute loss (Negative Log Likelihood for classification)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Function to evaluate the model's performance on validation/test set
def evaluate(model, data):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculations
        logits = model(data)  # Forward pass to get logits
        preds = logits.argmax(dim=1)  # Get the index of the maximum logit (predicted class)

        # Calculate accuracy for train, validation, and test sets
        train_acc = accuracy_score(data.y[data.train_mask].cpu(), preds[data.train_mask].cpu())
        val_acc = accuracy_score(data.y[data.val_mask].cpu(), preds[data.val_mask].cpu())
        test_acc = accuracy_score(data.y[data.test_mask].cpu(), preds[data.test_mask].cpu())

    print(f'Train Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}')

    return train_acc, val_acc, test_acc


In [11]:
from torch.optim import Adam
from sklearn.metrics import accuracy_score
optimizer = Adam(gat_model.parameters(), lr=0.005, weight_decay=5e-4)

In [12]:
# Train the model
train(gat_model, optimizer, data, epochs=200)

# Evaluate the model performance
evaluate(gat_model, data)

Epoch 0, Loss: 1.9639253616333008
Epoch 10, Loss: 1.8576102256774902
Epoch 20, Loss: 1.6442643404006958
Epoch 30, Loss: 1.151266098022461
Epoch 40, Loss: 0.768291175365448
Epoch 50, Loss: 0.6253531575202942
Epoch 60, Loss: 0.48335957527160645
Epoch 70, Loss: 0.4869670867919922
Epoch 80, Loss: 0.48715198040008545
Epoch 90, Loss: 0.34549281001091003
Epoch 100, Loss: 0.4010072350502014
Epoch 110, Loss: 0.4543887674808502
Epoch 120, Loss: 0.5482780933380127
Epoch 130, Loss: 0.2961910665035248
Epoch 140, Loss: 0.26287946105003357
Epoch 150, Loss: 0.3747282922267914
Epoch 160, Loss: 0.43459755182266235
Epoch 170, Loss: 0.28012654185295105
Epoch 180, Loss: 0.26574862003326416
Epoch 190, Loss: 0.4761897921562195
Train Accuracy: 1.0000, Validation Accuracy: 0.7660, Test Accuracy: 0.7760


(1.0, 0.766, 0.776)

In [13]:
train_acc, val_acc, test_acc = evaluate(gat_model, data)

Train Accuracy: 1.0000, Validation Accuracy: 0.7660, Test Accuracy: 0.7760
